In [ ]:
#import my functions
import helpers.fileHandler as fh
import utils.timePeriod as tp
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index

In [2]:
import xarray
import numpy


from eofs.xarray import Eof

In [1]:
def anoms(xr):
    '''Calculate an anomaly for the provided, using 1900 to 2000 climatology'''
    
    mean=xr.where((xr.time.dt.year>=1900) * (xr.time.dt.year<2000), 
                             drop=True).mean(dim='time')
    return xr-mean

In [ ]:
def sstAnoms(tsXr):
    ''' This is how Wang/Cai calculate anomalies'''
    
    #Area of interest for ec Indeces
    tsXr=tsXr.where(
        (tsXr.lat>=-15) & (tsXr.lat<=15) &
        (tsXr.lon>=140) & (tsXr.lon<=280),
        drop=True
    )
    
    #Rechunk so that time is all in one chunk. Not sure this is useful but it does seem to reduce memory needs
    tsXr=tsXr.chunk(-1, 'auto', 'auto')
    
    #Fit a quadratic and detrend using it
    trendXr = tsXr.polyfit('time', 2)
    trendXr = xarray.polyval(tsXr.time, trendXr.polyfit_coefficients, 'degree')
    detrendXr=tsXr-trendXr
    
    #calculate monthly anoms.
    sstAnomXr=detrendXr.groupby('time.month').apply(anoms)
    return sstAnomXr
    


In [ ]:
def eofSolver(sstAnomXr): 
    
    sstAnomXr.load()
    
    weights = numpy.cos(numpy.deg2rad(sstAnomXr.lat)
                ).values[..., numpy.newaxis]
    
    return Eof(sstAnomXr) #, weights=weights)

In [ ]:
def ecIndex(sstAnomXr):
    import numpy.polynomial as poly
    
    
    solver=eofSolver(sstAnomXr)
    pcTimeXr=solver.pcs(npcs=2, pcscaling=1)
    
    djfAnomXr=tp.averageForTimePeriod(
            sstAnomXr.to_dataset(name='enso')).rename({'year':'time'}).enso

    solver=eofSolver(djfAnomXr)
    djfPcXr=solver.pcs(npcs=2, pcscaling=1)
    
    
    pc1 = pcTimeXr.sel(mode=0)
    pc2 = pcTimeXr.sel(mode=1)
    
    pFit = poly.Polynomial.fit(pc1, pc2, 2)
    alpha = pFit.convert().coef[2]
    
    eofsXr = solver.eofs(eofscaling=1, neofs=2)
    
    pFitDjf = poly.Polynomial.fit(djfPcXr.sel(mode=0), djfPcXr.sel(mode=1), 2)
    alphaDjf = pFitDjf.convert().coef[2]
    
    cXr=(pcTimeXr.sel(mode=0)+pcTimeXr.sel(mode=1))/numpy.sqrt(2)
    eXr=(pcTimeXr.sel(mode=0)-pcTimeXr.sel(mode=1))/numpy.sqrt(2)
        
    indeces = xarray.merge([pcTimeXr.sel(mode=0).drop('mode').rename('pc1'),
                            pcTimeXr.sel(mode=1).drop('mode').rename('pc2'), 
                            eXr.rename('eIndex'),
                            cXr.rename('cIndex')])
    indeces['alpha']=alpha
    indeces['alphaDjf']=alphaDjf
    
    return indeces, pFit, eofsXr

In [ ]:
def ensoPlotter(da, ax):
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import matplotlib.pyplot as plt 
    import numpy
    
    precContours=numpy.arange(-0.6,0.61,0.1)
    
    cs=plt.contourf(da.lon, da.lat, da.values, #precContours, 
                    transform=ccrs.PlateCarree(), cmap='coolwarm', #extend='both' ,
                   center=True)

    gl=ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle=':')
    ax.coastlines(color='black')
    gl.top_labels=False
    #gl.left_labels=False

        #bottom legend
    cbar=plt.colorbar(orientation='horizontal', fraction=0.05, pad=0.05)